In [1]:
import pandas as pd
import re
import nltk
import pickle
import os
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive

# 1. Montage Drive
drive.mount('/content/drive')
nltk.download('stopwords')

# Configuration des chemins
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/projet_thinking/data_of_all_films.csv"
MODEL_DIR = "/content/drive/MyDrive/Colab Notebooks/projet_thinking/models/"
if not os.path.exists(MODEL_DIR): os.makedirs(MODEL_DIR)

# 2. Chargement
df = pd.read_csv(DATA_PATH)
print(f"📄 Dataset chargé : {len(df)} films.")
df = df.dropna(subset=['description']).reset_index(drop=True)
# 3. Nettoyage & Feature Engineering
def clean_text(text):
    if not isinstance(text, str): return ""
    text = text.lower()
    # Support des accents français
    text = re.sub(r'[^a-zàâçéèêëîïôûù\s]', '', text)
    stop_words = set(stopwords.words('french'))
    return " ".join([w for w in text.split() if w not in stop_words and len(w) > 2])

print("🧼 Préparation de la 'Metadata Soup'...")
df['cleaned_desc'] = df['description'].apply(clean_text)
df['cleaned_genres'] = df['genres'].apply(lambda x: clean_text(str(x)))

# On crée la soupe : Description + (Genres x 2) pour donner du poids au genre
df['soup'] = df['cleaned_desc'] + " " + (df['cleaned_genres'] + " ") * 2

# 4. Normalisation du Succès
# On normalise la popularité entre 0 et 1
scaler = MinMaxScaler()
df['norm_popularity'] = scaler.fit_transform(df[['popularity']].fillna(0))

# 5. Vectorisation TF-IDF
print("🧠 Entraînement TF-IDF sur la soupe...")
vectorizer = TfidfVectorizer(
    max_features=20000,
    ngram_range=(1, 3), # On monte à 3 pour capter les expressions complexes
    min_df=2,
    sublinear_tf=True
)
tfidf_matrix = vectorizer.fit_transform(df['soup'])

# 6. Sauvegarde des artefacts
print("💾 Sauvegarde...")
pickle.dump(vectorizer, open(MODEL_DIR + "tfidf_vectorizer.pkl", "wb"))
pickle.dump(tfidf_matrix, open(MODEL_DIR + "tfidf_matrix.pkl", "wb"))
# On garde les colonnes nécessaires pour le dashboard
df[['tmdbId', 'title', 'genres', 'roi', 'popularity', 'norm_popularity']].to_csv(MODEL_DIR + "metadata_simulator.csv", index=False)

print("✅ Modèle hybride prêt !")

Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


📄 Dataset chargé : 26782 films.
🧼 Préparation de la 'Metadata Soup'...
🧠 Entraînement TF-IDF sur la soupe...
💾 Sauvegarde...
✅ Modèle hybride prêt !


In [3]:
def simulate_smart_pitch(user_pitch, top_n=10):
    # 1. Calcul de la similarité textuelle
    cleaned_input = clean_text(user_pitch)
    query_vec = vectorizer.transform([cleaned_input])

    # Calcul de la similarité cosinus pure (renvoie un array numpy)
    text_sim = cosine_similarity(query_vec, tfidf_matrix).flatten()

    # 2. Score Hybride
    # On utilise .values pour s'assurer qu'on manipule des arrays numpy et éviter les conflits d'index
    pop_scores = df['norm_popularity'].values
    hybrid_score = text_sim * (1 + 0.5 * pop_scores)

    # 3. Top N indices
    top_indices = hybrid_score.argsort()[-top_n:][::-1]

    print(f"\n🚀 Résultats pour : '{user_pitch[:50]}...'")

    # Correction de l'erreur KeyError : on accède à l'index de l'array numpy
    best_score = hybrid_score[top_indices[0]]

    if best_score < 0.05:
        print("⚠️ Aucun film n'est vraiment proche. Voici les résultats les plus probables :")

    for i in top_indices:
        # On utilise .iloc pour être sûr de prendre la i-ème ligne physique du DataFrame
        row = df.iloc[i]
        real_sim = text_sim[i]
        score_final = hybrid_score[i]

        print(f"- {row['title']} | Score: {score_final:.2%} | (Sim Texte: {real_sim:.2%}) | Genre: {row['genres']} | Poster: {row['poster_url']}")

# Test
simulate_smart_pitch("Dans un futur proche, la ville de Los Angeles est en proie à une montée de la violence. Le détective privé Chris Raven est accusé du meurtre de sa femme. Il est jugé par la juge Maddox, une intelligence artificielle avancée du tribunal expéditif de Mercy Capital, auquel Chris a grandement contribué. Il a 90 minutes pour prouver lui-même son innocence. Après ce délai, l'IA déterminera son sort et le condamnera à mort")


🚀 Résultats pour : 'Dans un futur proche, la ville de Los Angeles est ...'
- The Two Jakes | Score: 18.22% | (Sim Texte: 18.08%) | Genre: Crime|Drama|Mystery|Romance | Poster: https://image.tmdb.org/t/p/w500/kmCxufDNl1hBW5YAXxLpKNNV6t3.jpg
- Everybody Wins | Score: 17.80% | (Sim Texte: 17.74%) | Genre: Crime | Poster: https://image.tmdb.org/t/p/w500/9LbugTDuQv98EB2K2uXYSHauHHa.jpg
- Dead Again | Score: 16.80% | (Sim Texte: 16.67%) | Genre: Mystery|Drama|Crime|Thriller|Romance | Poster: https://image.tmdb.org/t/p/w500/2rVr2hKmT411MDr60ml8Q0mB4wX.jpg
- Who Framed Roger Rabbit | Score: 16.34% | (Sim Texte: 15.79%) | Genre: Fantasy|Animation|Comedy|Crime | Poster: https://image.tmdb.org/t/p/w500/fJ1wJ9avrGj9VViC94CcRKL8N7w.jpg
- Chinatown | Score: 15.16% | (Sim Texte: 14.70%) | Genre: Crime|Drama|Mystery|Thriller | Poster: https://image.tmdb.org/t/p/w500/9chbM7rbh6o0Ok6v6mcT0a3GN9E.jpg
- Oldboy | Score: 14.60% | (Sim Texte: 14.28%) | Genre: Drama|Thriller|Mystery|Action | Poster: https://